# NB3: Notebook created for processing judgment text

**Output:** Clean dataset with the following variables. 

- [ ] `decision_date`: application date. 
- [ ] `referral_date `: date when the reference for preliminary referenc was sent. 
- [ ] `referring_court_name`: ???
- [ ] `full_text`: 
- [ ] `par_location`:
- [ ] 

**Output**: Create script that scrapes updates

## Environment and Settings for Python

In [74]:
# REQUIRED  Libraries
import os 
import sys
import numpy as np
import pandas as pd
import glob 
#import pyarrow.parquet as pq
#import pyarrow
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

# ADD project paths for for easier access across whole project
#paths_script_path = "C:\\Users\\mauricmm\\iCloudDrive\\cloudgit\\uio24emc\\"
parent_dir = os.path.dirname(os.getcwd(),) +"\\"
sys.path.append(parent_dir)
import paths

# Access paths globally
paths.figures_dir
# Import the functions from custom scripts
from  costum_py import extract_questions

## Start Processing Judgements Text

In [4]:
#READ IUROPA Text Corpus Data
iuropa_corpus_path = "C:\\gitprojects\\uio24emc\\raw\\iuropa_text.gz.parquet"
iuropa_corpus = pq.read_table(iuropa_corpus_path)
iuropa_corpus_df = iuropa_corpus.to_pandas()
schema = pq.read_schema(iuropa_corpus_path)

NameError: name 'pq' is not defined

In [ ]:
# [ ] CREATE columns selector 
# [ ] CREATE rows boolean mask
# [ ] CHECK if the text from the sections is good  to process. 
# [ ] OUTPUT target dataset to to data as  "judgements" 
iuropa_corpus_df.columns


# Columns selector Selector
cols_selector =  ['document_id', 'paragraph_id', 'language', 'ecli', 'court',
       'date', 'year', 'text', 'line_id', 'line_id_prop', 'section',
       'paragraph_type', 'paragraph_number', 'nchar', 'html_class',
       'html_attr']

# BOOLEAN Conditions for mask



In [ ]:
iuropa_corpus_df.head()
iuropa_corpus_df

# Search for rows where the specified column contains the search text
# This example performs a case-insensitive search
limited_df = iuropa_corpus_df.head(1000)
matching_rows = limited_df[limited_df['text'].str.contains( "admissibility" , case=False, na=False)]

### NOTES

In [ ]:
# USE Query for complex but not to elaborated queries
filtered_df = df.query('description.str.contains("data") & id > 1', engine='python')

# USE BOOLEAN masks for more complex conditions
condition1 = df['description'].str.contains('data', case=False, na=False)
condition2 = df['age'] > 30
condition3 = df['name'].isin(['Alice', 'David'])
# Combine the conditions using logical operators
complex_mask = condition1 & condition2 | condition3

# Apply the mask to filter the DataFrame
filtered_df = df[complex_mask]

## NB1 while it finishes

In [18]:
#######################################
# NB1 Applications Scaping
#######################################
# IGNORE replicatses data workiong with over there
#######################################
iu_uio24emc_dir = f"{paths.raw_dir}iu_uio24emc"
## LOAD data for 
apps_ds = pd.read_csv(f"{iu_uio24emc_dir}\\apps_scraping.csv")
## Subset for 2008-2023 
apps_ds =apps_ds[apps_ds['eurlex_available']==1]


In [82]:
#SELECT folder path for easier writing filenames
apps_txt_dir = f"{paths.raw_dir}appstxts"

print

['c:\\gitprojects\\uio24emc\\raw\\appstxts\\ref_2008_0001it.txt', 'c:\\gitprojects\\uio24emc\\raw\\appstxts\\ref_2008_0002it.txt']


In [83]:
#######################################
# PROCESSES Convert *.txt to clean *.txt
#######################################
#############################
# OUTPUT ./raw dyad_apps_citations UoM Level 
# dyad_apps_text['referral_par_id'] = dyad_apps_text.apply(lambda row: f"{row['uoa_dyad_id']}_PAR{str(row['par_num']).zfill(3)}", axis=1)    :EX: REF_2010_0242IT_PAR01
import html2text
from bs4 import BeautifulSoup 

#SELECT folder  path for easier file accessing 
apps_txt_dir = f"{paths.raw_dir}appstxts"

all_txts_paths = glob.glob(f"{apps_txt_dir}\\*.txt")[0:20]

#SELECT folder path for easier file wirting
apps_clean_txt_dir = f"{paths.raw_dir}cleanappstxt"

for txt_i in all_txts_paths:
    #EXTRACT base file name and create clean file file name
    basename = os.path.basename(txt_i)
    out_path = f"{apps_clean_txt_dir}\\{basename}"
    if os.path.exists(out_path):
        print(f"File already exists: {txt_i}")
    else:
        #READ file path into current machine
        with open(txt_i, "r", encoding="utf-8") as in_file:
            in_text_content = in_file.read()
           #PARSE text into with html2text
            h2t = html2text.HTML2Text()
            h2t.ignore_links = True
            out_text = h2t.handle(in_text_content)
        # Write the parsed text to the output file
        with open(out_path, "w", encoding="utf-8") as file:
            file.write(out_text)

#######################
# DONE Just copy paste to NB1
#######################

File already exists: c:\gitprojects\uio24emc\raw\appstxts\ref_2008_0005dk.txt
File already exists: c:\gitprojects\uio24emc\raw\appstxts\ref_2008_0012be.txt
File already exists: c:\gitprojects\uio24emc\raw\appstxts\ref_2008_0013de.txt


In [67]:
#######################################
#FUNCTION Parse plain text to dataframe at 1DF UoM level
#######################################
#############################
# OUTPUT ./raw/dyad_apps_text.csv UoM Level 

import html2text
from bs4 import BeautifulSoup 
import requests
import re

apps_clean_txt_dir = f"{paths.raw_dir}cleanappstxt"
# Regular expression pattern to match the Markdown table
table_pattern = r"(\|.*\|(?:\n\|[-:]*\|)+\n(?:\|.*\|(?:\n)?)*)"

doc1 = "C:\\gitprojects\\uio24emc\\raw\\cleanappstxt\\ref_2008_0005dk.txt"
doc2 = "C:\\gitprojects\\uio24emc\\raw\\cleanappstxt\\ref_2010_0240de.txt"
all_txts_paths = [doc1, doc2]


for txt_i in all_txts_paths:
    #KEY_ID column name and create clean file file name
    uio_dyad_id = ((os.path.basename(txt_i))[:-4]).upper()
    #LOAD file path into current machine as file
    with open(txt_i, "r", encoding="utf-8") as file:
        #READ Lines
        lines = file.readlines()
        print(lines[27:])
        data = []
        
    break


################
# ONCE table exists
# CREATE Unique id 
# dyad_apps_text['referral_par_id'] = dyad_apps_text.apply(lambda row: f"{row['uoa_dyad_id']}_PAR{str(row['par_num']).zfill(3)}", axis=1)    :EX: REF_2010_0242IT_PAR01


(i) |  Can the storing and subsequent printing out of a text extract from an article in a daily newspaper, consisting of a search word and the five preceding and five subsequent words, be regarded as acts of reproduction which are protected (see Article 2 of the Infosoc Directive (1))?  



In [50]:
####################################
# FUNCTION Beautiful soup extracts and puts into column
####################################
apps_ds =apps_ds[apps_ds['eurlex_available']==1]

#In case you want to RE RUN IT
baselink_txtonly ="https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:"
text_regex = re.compile(r'Question(s)? referred', re.IGNORECASE)

for index, row in apps_ds.iterrows():
    dyad_id = row['uoa_dyad_id']
    celex = str( row['apps_celex'])
    baselink ="https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:"
    # Create the URL and filename dynamically using string formatting
    url_i = f"{baselink}{celex}"
    response = requests.get(url_i)
    # Parse the webpage content
    soup = BeautifulSoup(response.content, 'html.parser')
    referred_text = soup.find(text=text_regex)
    # Initialize a variable to hold the table DataFrame
    table_df = None
    if referred_text:
        # Find the next table after the found text
        table = referred_text.find_next('table')
        if table:
            # Convert the table to a DataFrame
            table_df = pd.read_html(str(table))[0]
            # Save the table to a CSV file (optional)
            table_df.to_csv('referred_question_table.csv', index=False)
            
            print('Extracted Table:\n', table_df)
        else:
            print("No table found after the referred text.")
    else:
        print("The specified text was not found on the page.")
        
        # Optional: Save each table to a CSV file
        break
    break 



No table found after the referred text.


C:\Users\mauricmm\AppData\Local\Temp\ipykernel_13856\163123204.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  referred_text = soup.find(text=text_regex)


In [53]:
#TEST extracting

    

In [ ]:
############################
# FUNCTION Get celexes from apps
#############################
# OUTPUT ./raw dyad_apps_citations UoM Level 
# dyad_apps_citations['referral_par_id'] = dyad_apps_citations.apply(lambda row: f"{row['uoa_dyad_id']}_PAR{str(row['par_num']).zfill(3)}", axis=1)    :EX: REF_2010_0242IT_PAR01

# Same as above h2t.ignore_links = False
# extract celexes  

#FUNCTION Fuzzy Match 
# Subset citations in c in C = XofC
# ITERATE{ through every par 
# items =  
# ITERATE{ through every citation match  if
# Alternative is Create a citations lookup table with names and celexes 
# #####################################################
# #######################################################
#   #READ file content 
#         content = file.read()
#     # Find the first Markdown table in the content
#     match = re.search(table_pattern, content)
#     if match:
#         markdown_table = match.group(1)
#         #df = pd.read_csv(pd.compat.StringIO(markdown_table), sep="|", skipinitialspace=True)
#     print(markdown_table)    

In [8]:
# NOTE might be useful
# CREATE Unique id 
# dyad_apps_text['referral_par_id'] = dyad_apps_text.apply(lambda row: f"{row['uoa_dyad_id']}_PAR{str(row['par_num']).zfill(3)}", axis=1)    :EX: REF_2010_0242IT_PAR01

# doc1 = "C:\\gitprojects\\uio24emc\\raw\\appstxts\\ref_2008_0005dk.txt"
# with open(doc1, 'r') as file:
#     # Read the entire content
#     content = file.read()

# # Print the content
# print(content)

<?xml version="1.0" encoding="UTF-8"?>

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML//EN" "xhtml-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">

<!-- CONVEX # converter_version:3.2 # generated_on:20140122-2224 -->

<head><meta name="format-detection" content="telephone=no"/>
   <meta http-equiv="content-type" content="text/html; charset=utf-8"/>
   <script type="text/javascript" src="/ruxitagentjs_ICANVfgqrux_10289240325103055.js" data-dtconfig="app=47d4c64c3b67ec69|agentId=8a988541f17fe749|featureHash=ICANVfgqrux|rdnt=1|uxrgce=1|bp=3|cuc=m097nmfl|mel=100000|mb=null|dpvc=1|iub=null|ssv=4|lastModification=1723033979053|tp=500,50,0|agentUri=/ruxitagentjs_ICANVfgqrux_10289240325103055.js|reportUrl=/rb_39a3e95b-5423-482c-879b-99ef235dffeb|rid=RID_682125163|rpid=39833237|domain=europa.eu"></script><link type="text/css" rel="stylesheet"
         href="./../../../../css/oj/oj.css"/>
   <title>C_2008064EN.01002802.xml</title>
<link rel="canonical" href="https://eur-lex.europa.eu/legal-c